In [26]:
from _helper_basics_ import *
from _helper_gender_ import *
%reload_ext autoreload

In [1]:
import sys
## To get helper functions from another folder
# sys.path.insert(0, '../') # if _helper_basics_ is in previous folder
now_i_am_at = 'home' # home dso test
if now_i_am_at=='home': sys.path.insert(0, 'E:/Leonard HDD/Dropbox/DSO/Tasks/')
elif now_i_am_at=='dso': sys.path.insert(0, 'D:/Dropbox/DSO/Tasks')

from _helper_basics_ import *
from _helper_gender_ import *

%matplotlib inline
plt.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

print('keras',      keras.__version__)
print('tensorflow', tf.__version__)

Using TensorFlow backend.


keras 2.0.8
tensorflow 1.3.0


In [2]:
yf = np.asarray([0,1])
ym = np.asarray([1,0])


nfft = 256   # sr=8000
# nfft = 512   # sr=16000
hop_length = nfft//3 # 256 = 30ms
num_freq,num_time,num_channel    = nfft//2+1,48,1 # 0.5 seconds speech is about 48 time_samples
print('nfft:{}'.format(nfft),'hop_length:{}'.format(hop_length),
      '\nnum_freq:{}'.format(num_freq),'num_time:{}'.format(num_time),'num_channel:{}'.format(num_channel))
Spect_Det = [nfft, hop_length, num_freq,num_time,num_channel]

nfft:256 hop_length:85 
num_freq:129 num_time:48 num_channel:1


In [3]:
pwd = os.getcwd()
print('pwd', pwd)

Dataset_dir = os.path.join(pwd,'..','..','..','Speech Audio Text','3) Dataset')
get_immediate_subdirectories(Dataset_dir)

pwd E:\Leonard HDD\Dropbox\Meetup\Deep Learning Developer Course\Personal2_Gender_Audio_Classification


['AMI',
 'BABEL_CORPUS',
 'Gender',
 'Google_Speech_Commands',
 'MUSAN',
 'Reverb_1',
 'Reverb_2',
 'RIRS_NOISES',
 'Speech MNIST',
 'Speeches',
 'Switchboard',
 'TIMIT',
 'Voice_Converision',
 'z.samples']

In [4]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

## Data_1
    Google Speech Command Dataset
    

## Data_2
    Downloaded in https://datashare.is.ed.ac.uk/handle/10283/2211
    Phrases spoken by different people
    But are not aligned as different speakers have different styles and length
    
    Currently v1_FCN is trained on this

In [76]:
Data_2_dir = os.path.join(Dataset_dir,'Voice_Converision', 'Data_2_Datashare','vcc2016_training')
dir_list = get_immediate_subdirectories(Data_2_dir) 
print('dir_list:',dir_list)

pwd = os.getcwd() # 'D:\\Dropbox\\Meetup\\Deep Learning Developer Course\\Personal2_Gender_Audio_Classification'
Data_tdt = os.path.join(pwd,'Data_tdt')
trg_dir = os.path.join(Data_tdt,'train')

## Male Female
dir_list_F,dir_list_M = [],[]
for i in dir_list:
    if 'F' in i: dir_list_F.append(i)
    elif 'M' in i: dir_list_M.append(i)
print('dir_list_F',dir_list_F)
print('dir_list_M',dir_list_M)

## ListW
TRG_list,VAL_list, TEST_list = [],[],[]
for i in range(len(dir_list)//2):
    if i==3:
        VAL_list.append(dir_list_F[i])
        VAL_list.append(dir_list_M[i])
    elif i==4:
        TEST_list.append(dir_list_F[i])
        TEST_list.append(dir_list_M[i])
    else:
        TRG_list.append(dir_list_F[i])
        TRG_list.append(dir_list_M[i])
trg_path_list,trg_list = data_2_tdt(TRG_list, Data_2_dir)
print('trg_path_list', len(trg_path_list))
val_path_list,val_list = data_2_tdt(VAL_list, Data_2_dir)
print('val_path_list', len(val_path_list))
test_path_list,test_list = data_2_tdt(TEST_list, Data_2_dir)
print('test_path_list', len(test_path_list))

dir_list: ['SF1', 'SF2', 'SF3', 'SM1', 'SM2', 'TF1', 'TF2', 'TM1', 'TM2', 'TM3']
dir_list_F ['SF1', 'SF2', 'SF3', 'TF1', 'TF2']
dir_list_M ['SM1', 'SM2', 'TM1', 'TM2', 'TM3']
trg_path_list 972
val_path_list 324
test_path_list 324


In [22]:
Y_train = dump_load_pickle(os.path.join(trg_dir,'Y_train'), 'load')
X_train = dump_load_pickle(os.path.join(trg_dir,'X_train'), 'load')
X_train = X_train.reshape(len(X_train), num_freq,num_time,num_channel)
print('Y_train', Y_train.shape)
print('X_train', X_train.shape)

Y_train (9947, 2)
X_train (9947, 129, 48, 1)


In [102]:
X_train_wav = dump_load_pickle(os.path.join(pwd,'Data_tdt','train','X_train_wav'), 'load')
X_train_lps = dump_load_pickle(os.path.join(pwd,'Data_tdt','train','X_train_lps'), 'load')
print('X_train_wav', len(X_train_wav))
print('X_train_lps', len(X_train_lps))

X_train_wav 972
X_train_lps 972


In [23]:
male_num,female_num = 0,0
for i in Y_train:
    if np.any(i==ym): male_num += 1
    else: female_num += 1
        
print('male_num',male_num)
print('female_num',female_num)

male_num 5046
female_num 4901


## Data_3
    Timit
    

In [110]:
TIMIT_Gender_Folder = os.path.join(pwd, 'TIMIT_Gender')

Data_3_dir = os.path.join(Dataset_dir,'TIMIT')
trg_dir  = os.path.join(Data_3_dir, 'timit_train')
trg_path_list = glob.glob( os.path.join(trg_dir, '*.wav') )
trg_list = []
for i in trg_path_list:
    trg_list.append( i[len(trg_dir)+1:] )

In [111]:
X_train  = dump_load_pickle(os.path.join(TIMIT_Gender_Folder,'train','X_train'), 'load')
Y_train  = dump_load_pickle(os.path.join(TIMIT_Gender_Folder,'train','Y_train'), 'load')
print('X_train',X_train.shape)
print('Y_train',Y_train.shape)

X_train (40739, 129, 48, 1)
Y_train (40739, 2)


In [112]:
X_train_wav = dump_load_pickle(os.path.join(TIMIT_Gender_Folder,'train','X_train_wav'), 'load')
X_train_lps = dump_load_pickle(os.path.join(TIMIT_Gender_Folder,'train','X_train_lps'), 'load')
print('X_train_wav',len(X_train_wav))
print('X_train_lps',len(X_train_lps))

X_train_wav 4619
X_train_lps 4619


# Load Model for prediction

#### V1 Simple ConvNet + Dense layers

In [40]:
Archi_dir = "v1_FCN/"
Weights_path = Archi_dir+"Logs/"
Ckpt_Mod_Weights_fold = Archi_dir+"Checkpoint_Model_Weights/"

model_path = Weights_path+Archi_dir[:-1]+"_model_v1_1.json"
weights_path = Weights_path+Archi_dir[:-1]+"_weights_v1_1.h5"
# weights_path = Ckpt_Mod_Weights_fold+"v1_1/"+"weights_v1_1_Epoch-0282_L-0.01.hdf5"

model_path = Weights_path+Archi_dir[:-1]+"_model_v1_2.json"
weights_path = Weights_path+Archi_dir[:-1]+"_weights_v1_2.h5"
# weights_path = Ckpt_Mod_Weights_fold+"v1_1/"+"weights_v1_1_Epoch-0282_L-0.01.hdf5"

v1 = load_model(model_path, 'path', weights_path)
v1.summary()

# adam_opt = Adam(lr=1e-3)
sgd_opt = SGD(lr=1e-3, momentum=0.9)
v1.compile(loss=categorical_crossentropy ,optimizer=sgd_opt, metrics=['categorical_accuracy'])

Loaded model from path : v1_FCN/Logs/v1_FCN_model_v1_2.json
Loaded weights         : v1_FCN/Logs/v1_FCN_weights_v1_2.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 129, 48, 1)        0         
_________________________________________________________________
0_Conv2D (Conv2D)            (None, 129, 48, 64)       640       
_________________________________________________________________
1_Conv2D (Conv2D)            (None, 129, 48, 64)       36928     
_________________________________________________________________
1_Pool (MaxPooling2D)        (None, 64, 24, 64)        0         
_________________________________________________________________
2_Conv2D (Conv2D)            (None, 64, 24, 64)        36928     
_________________________________________________________________
GlobalPool (GlobalAveragePoo (None, 64)                0         
______________________

In [ ]:
score_train = v1.evaluate(X_train, Y_train, batch_size=256)
# score_val = v1.evaluate(X_val, Y_val, batch_size=256)
# score_test = v1.evaluate(X_test, Y_test, batch_size=256)

print('score_train',score_train)
# print('score_val',score_val)
# print('score_test',score_test)

### Gender-Split Prediction

In [43]:
X_train_male,Y_train_male,X_train_female,Y_train_female = gender_split(X_train,Y_train)
# X_val_male,Y_val_male,X_val_female,Y_val_female = gender_split(X_val,Y_val)
# X_test_male,Y_test_male,X_test_female,Y_test_female = gender_split(X_test,Y_test)

min_num=11482, male_num=29257, female_num=11482, idx=40738
male_num 29257
female_num 11482


In [44]:
batch_size = 512
print('Predicting Male...');   score_male   = v1.evaluate(X_train_male,   Y_train_male,   batch_size=batch_size)
print('Predicting Female...'); score_female = v1.evaluate(X_train_female, Y_train_female, batch_size=batch_size)
# score_total = v1.evaluate(X_train, Y_train, batch_size=256)

print('score_male',score_male)
print('score_female',score_female)
# print('score_total',score_total)

11482/11482 [==============================] - 47s    
score_male [0.97660668753475877, 0.076767952967501718]
score_female [0.1509068370296282, 0.9999129048956954]


In [49]:
## Random Female Prediction
idx = np.random.randint(0,len(X_train_female))
yf = np.asarray([0,1])
ym = np.asarray([1,0])
## Prediction
y_hat_train = v1.predict(X_train_female[idx].reshape(1,num_freq,num_time,num_channel))
print('Predicted Female : {}%'.format(np.sum(y_hat_train*yf)*100.) )
print('Prediction: ', y_hat_train[0])
print('Truth     : ', Y_train_female[idx])

## Random Male Prediction
idx = np.random.randint(0,len(X_train_male))
yf = np.asarray([0,1])
ym = np.asarray([1,0])
## Prediction
y_hat_train = v1.predict(X_train_male[idx].reshape(1,num_freq,num_time,num_channel))
print('Predicted Male : {}%'.format(np.sum(y_hat_train*ym)*100.) )
print('Prediction: ', y_hat_train[0])
print('Truth     : ', Y_train_male[idx])

Predicted Female : 95.87957859039307%
Prediction:  [ 0.04120423  0.95879579]
Truth     :  [0 1]
Predicted Male : 42.107248306274414%
Prediction:  [ 0.42107248  0.57892752]
Truth     :  [1 0]


### Average of softmaxes

In [117]:
# gen_position = 1 # for Data 2
gen_position = 4 # for Data 3

count_correct = Average_Soft_Pred(v1, gen_position, X_train_lps, trg_list, Spect_Det)
print('Accuracy: {}/{} {}%'.format(count_correct,len(trg_list),100.*count_correct/len(trg_list)))

Finished 9.98051526304395%
Finished 19.9610305260879%


C:\Users\Leonard\Anaconda3\envs\virenv_py3_gpu\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Leonard\Anaconda3\envs\virenv_py3_gpu\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Finished 29.941545789131847%
Finished 39.9220610521758%
Finished 49.90257631521975%
Finished 59.883091578263695%
Finished 69.86360684130764%
Finished 79.8441221043516%
Finished 89.82463736739554%
Finished 99.8051526304395%
Accuracy: 1535/4619 33.23230136393159%


In [85]:
from _helper_basics_ import *
from _helper_gender_ import *
%reload_ext autoreload